In [4]:
%matplotlib inline

from torch.utils.data import Dataset, DataLoader
import torchvision.datasets as datasets
from tqdm.notebook import tqdm as tqdm
from torchvision import transforms

import matplotlib.pyplot as plt
import torch.nn.functional as F
from torch import utils
import torch.nn as nn
import numpy as np
import os, imageio
import pickle
import torch
import cv2

### Dataloader

In [5]:
mnist_trans = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(0, 1.0),
    transforms.CenterCrop((28))
])

In [6]:
mnist= datasets.MNIST(root='./../data', train=True,download=False, transform=mnist_trans)
mnist = torch.utils.data.random_split(mnist, [10000, len(mnist)-10000])[0] # gets split into two parts [10k and 50k] and we slice 0 index.


### Device

In [7]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

print('Using PyTorch version:', torch.__version__, ' Device:', device)

# torch.set_default_dtype(torch.float64)
torch.set_default_device(device)

Using PyTorch version: 2.1.0.dev20230622+cu121  Device: cuda


In [8]:
torch.set_default_dtype(torch.float64)
torch.set_default_device(device)

### Converting Dataset

In [9]:
def generate_pixel_coordinates(img):
    
    height, width, _ = img.permute(1,2,0).shape
    
    # Generate coordinates along the x-axis and y-axis
    x_coords = np.linspace(0, 1, width, endpoint=False)
    y_coords = np.linspace(0, 1, height, endpoint=False)
    
    # Create a meshgrid of coordinates
    x_mesh, y_mesh = np.meshgrid(x_coords, y_coords)
    
    # Stack the coordinates and reshape to obtain the final output
    coordinates = np.stack([x_mesh, y_mesh], axis=-1)
    
    return coordinates

### Fourier Feature mapping


In [10]:
def input_mapping(x, B):
    '''
        cos and sin of input are joined together to increased the dimension to 4.
    '''
    if B is None:
        return x
    else:
        B = B.to(x)
        x_proj = (2.*np.pi*x) @ B.T #512,2
        return torch.cat([torch.sin(x_proj), torch.cos(x_proj)], dim=-1)    #  2*len(B) #512,4

In [11]:
def batch_loader(coord_input,image,map_dict,mapping = 'none'):

    coord_input = input_mapping(coord_input, map_dict[mapping])
    
    test_data = [coord_input, image]
    train_data = [coord_input[::2], image[::2]]
    
    train_x = torch.tensor(train_data[0])#.reshape(-1,coord_input.shape[2]) # because input has 4 dimension
    train_y = torch.tensor(train_data[1])#.reshape(-1,3)
    test_x = torch.tensor(test_data[0])#.reshape(-1,coord_input.shape[2]) # because input has 4 dimension
    test_y = torch.tensor(test_data[1])#.reshape(-1,3)

    return train_x,train_y,test_x,test_y

### MLP network

In [12]:
class Net(nn.Module):
    def __init__(self,input_dim):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_dim, 50) # input has 4 dimensions.
        self.fc1_drop = nn.Dropout(0.2)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(50, 50)
        self.fc2_drop = nn.Dropout(0.2)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(50,1)

    def forward(self, x):
        # x = x.view(-1,4) # input has 4 dimensions.
        x = self.relu1(self.fc1(x))
        x = self.fc1_drop(x)
        x = self.relu2(self.fc2(x))
        x = self.fc2_drop(x)
        return self.fc3(x)

## Train

In [10]:
# train_x,train_y,test_x,test_y = batch_loader(xy_grid,x.permute(1,2,0),B_dict,keys)

In [11]:
# input = test_x.to(device)
# target = test_y.to(device)

# model = Net(input.shape[2]).to(device)
# optimizer = torch.optim.Adam(list(model.parameters()), lr=1e-2)


# for epoch in tqdm(range(1501)):
#     optimizer.zero_grad()

#     generated = model(input)

#     loss = torch.nn.functional.l1_loss(target, generated)

#     loss.backward()
#     optimizer.step()

#     if epoch % 100 == 0:
#       print('Epoch %d, loss = %.03f' % (epoch, float(loss)))
#       plt.imshow(generated.detach().cpu().numpy(),cmap='gray')
#       plt.show()

In [12]:
def model_weights(coordinates,image,img_no,dictionary,mapping,save = False):

    train_x,train_y,test_x,test_y = batch_loader(coordinates,image.permute(1,2,0),dictionary,mapping)

    input = test_x.to(device)
    target = test_y.to(device)

    model = Net(input.shape[2]).to(device)
    optimizer = torch.optim.Adam(list(model.parameters()), lr=1e-2)


    for epoch in range(1501):
        optimizer.zero_grad()

        generated = model(input)

        loss = torch.nn.functional.l1_loss(target, generated)

        loss.backward()
        optimizer.step()

        # if epoch % 300 == 0:
        #     print('Epoch %d, loss = %.03f' % (epoch, float(loss)))
        #     plt.imshow(generated.detach().cpu().numpy(),cmap='gray')
        #     plt.show()

    if save:
        torch.save(model.state_dict(), '../weights2/{fname}.pth'.format(fname=img_no))
    
    torch.cuda.empty_cache()
    del model
    

In [13]:
mapping_size = 64

B_dict = {}

B_gauss = torch.normal(0,1,size=(mapping_size,2))
for scale in [10.]:
    B_dict[f'gauss_{scale}'] = B_gauss * scale

for k in B_dict:
    keys = k

In [14]:
x,_ = mnist[0]

xy_grid = torch.from_numpy(generate_pixel_coordinates(x))

In [15]:
labels = [mnist[i][1] for i in range(len(mnist))]

In [16]:
torch.save(mnist,"../data/mnist.pt")

with open("../data/labels", "wb") as fp:   #Pickling
  pickle.dump(labels, fp)

In [17]:
for index in tqdm(range(len(mnist))):

    model_weights(xy_grid,mnist[index][0],index,B_dict,keys,save=True)

  0%|          | 0/10000 [00:00<?, ?it/s]

c:\MLAI\lib\site-packages\torch\utils\_device.py:76: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return func(*args, **kwargs)


### Load model

In [23]:
def flatten_weights(img_no):

  path = 'C:/Users/ayush/OneDrive/Desktop/UoE/Dissertation/Implicit Networks/weights2/{fname}.pth'.format(fname=img_no)

  model.load_state_dict(torch.load(path))

  weights = []

  for param in model.parameters():

    weights.extend(list(torch.flatten(param.data).detach().cpu().numpy()))

  return np.array(weights)


In [24]:
model = Net(128)
weights_dict = {}

for i in range(10000):

  weights_dict[str(i)] = flatten_weights(i)

In [25]:
import pickle 

with open('C:/Users/ayush/OneDrive/Desktop/UoE/Dissertation/Implicit Networks/data/weights_dictionary.pkl', 'wb') as f:
    pickle.dump(weights_dict, f)

In [27]:
weights_dict['9999'].shape

(9051,)

In [35]:
plt.imshow(weights_dict['9999'].resize((1,28,28)))

ValueError: cannot resize an array that references or is referenced
by another array in this way.
Use the np.resize function or refcheck=False